In [1]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler
from nitime import algorithms as alg

c:\anaconda3\lib\site-packages\pandas\__init__.py:7: DeprecationWarning: bad escape \s
  from pandas import hashtable, tslib, lib
c:\anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
c:\anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
c:\anaconda3\lib\site-packages\ipykernel\pylab\config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
c:\anaconda3\lib\site-packages\ipykernel\pylab\config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
c:\anaconda3\lib\site-packages\ipykernel\pylab\

In [2]:
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()
EDFfiles

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final_annotations.txt', 'suj9_d2final.edf', 'suj9_d2final_annotations.txt', 'testing-montage-2.mtg', 'Tr

c:\anaconda3\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
c:\anaconda3\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


['s5d2_final.edf',
 's6n2_final.edf',
 'suj10_d1final.edf',
 'suj10_d2final.edf',
 'suj13_l2nap_day2 edited.edf',
 'suj13_l2nap_day2 edited1.edf',
 'suj5_d1final.edf',
 'suj6_d1final.edf',
 'suj8_d1final.edf',
 'suj8_d2final.edf',
 'suj9_d1final.edf',
 'suj9_d2final.edf']

In [3]:
# initialization and preallocation
TrainingLabel=[]
TrainingData = []
totoal_lenght = 0
Fs=1000
    
# storing necessary features: raw data, low pass   
for idx,item in enumerate(EDFfiles):
    if item == 'suj13_l2nap_day2 edited.edf' or item =='suj13_l2nap_day2 edited1.edf':
        pass
    else:
        file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=idx)
        # channel list for artifact(eyemovement or chewing) removal - ROC and/or LOC
        channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
        raw = eegPinelineDesign.load_data(file_to_read,channelList,None, 100)# low pass
        annotation_to_read = [x for x in Annotationfiles if fileName in x]
        file = pd.read_csv(annotation_to_read[0])
        windowsLabel = eegPinelineDesign.annotation_to_labels(Annotationfiles,fileName)
        # parameterize annotations; use single number because the training output has one node only
        # put a temporal label storage for counting
        # channel list for extracting segments
        # since the label is only one per training data segment, I add up the segments across all six channels
        tempTrainingLabel=[]
        channelList = ['F3','F4','C3','C4','O1','O2']
        raw.pick_channels(channelList)
        totoal_lenght += len(windowsLabel)
        print('length of labels',totoal_lenght)
        for sample,items in enumerate(windowsLabel):
            if items not in ['w','1','2','3']:
                print('not putting in')
            else:
                try:
                    if items == 'w':
                        TrainingLabel.append(0)
                
                    elif items == '1':
                        TrainingLabel.append(1)
                
                    elif items == '2':
                        TrainingLabel.append(2)
                
                    elif items == '3':
                        TrainingLabel.append(3)
                    #print('done labeling')
                    for index,chan_names in enumerate(['F3','F4','C3','C4','O1','O2']):
                        if index == 0:
                            startPoint = sample*30;endPoint = (sample+1)*30
                            start,stop = raw.time_as_index([startPoint,endPoint])
                            tempSegment,time = raw[index,start:stop]
                            tempdata = tempSegment
                        else:
                            startPoint = sample* 30;endPoint = (sample+1)*30
                            start,stop = raw.time_as_index([startPoint,endPoint])
                            tempSegment, time = raw[index,start:stop]
                            tempdata += tempSegment
                    #print('done segments')
                    band_data = []
                    delta_band = mne.filter.low_pass_filter(tempdata,Fs,4)
                    theta_band = mne.filter.band_pass_filter(tempdata,Fs,4,7.5)
                    alpha_band = mne.filter.band_pass_filter(tempdata,Fs,7.5,14)
                    beta_band  = mne.filter.band_pass_filter(tempdata,Fs,14,40)
                    gamma_band = mne.filter.high_pass_filter(tempdata,Fs,40)
                    #print('done band pass')
                    band_data.append([delta_band,theta_band,alpha_band,beta_band,gamma_band])
    
                    before_data_to_store=[]
                    for freq_band in band_data[0]:
                        coef,sigma = alg.AR_est_LD(np.array(freq_band)[0,:],2)
                        before_data_to_store.append(np.append(coef,sigma))
                    TrainingData.append(list(np.array(before_data_to_store).ravel()))
                    #print(len(TrainingLabel),len(TrainingData))
                
                except:
                    print('skip this instance')
                    if len(TrainingLabel) != len(TrainingData):
                        if len(TrainingLabel) > len(TrainingData):
                            TrainingLabel.pop()
                        elif len(TrainingLabel) > len(TrainingData):
                            TrainingData.pop()
                
        
        
            

Extracting edf Parameters from s5d2_final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3383999  =      0.000 ...  3383.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    5, 6, 0, 0, 7
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 4 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Low-pass filtering at 5e+02 Hz
Low-pass filtering at 1e+02 Hz
length of labels 113
Extracting edf Parameters from s6n2_final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 2073999  =      0.000 ...  2073.999 secs...
[

c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 4.5Hz is only 17.2dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 8.0Hz is only 15.6dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 7.0Hz is only 15.6dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 13.5Hz is only 15.6dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 39.5Hz is only 16.9dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 4.5Hz is only 11.6dB.
  '%0.1fdB.' % (att_freq, att_db))


length of labels 304
Extracting edf Parameters from suj10_d2final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3704999  =      0.000 ...  3704.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 3 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    1, 4, 6, 4, 4, 0
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 4 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Low-pass filtering at 5e+02 Hz
Low-pass filtering at 1e+02 Hz
length of labels 429
not putting in
not putting in
not putting in
not putting in
not putting in
skip this instance
Extracting edf Parameters from suj5_d1final.edf...
S

c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 3.5Hz is only 8.9dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 14.5Hz is only 8.9dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 13.5Hz is only 9.0dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 39.5Hz is only 11.7dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 3.5Hz is only 19.7dB.
  '%0.1fdB.' % (att_freq, att_db))


length of labels 1027
skip this instance
skip this instance
Extracting edf Parameters from suj9_d2final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 2589999  =      0.000 ...  2589.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 1 artifact by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    6, 3, 3, 6
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 2 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Low-pass filtering at 5e+02 Hz
Low-pass filtering at 1e+02 Hz
length of labels 1114
not putting in
not putting in
not putting in


c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 14.5Hz is only 19.7dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 40.5Hz is only 19.7dB.
  '%0.1fdB.' % (att_freq, att_db))


In [4]:
#clf = OneVsRestClassifier(SVC(kernel='linear',tol=1e-6,max_iter=5e5))
clf = OneVsRestClassifier(LogisticRegression(penalty='l1',C=5,tol=10e-9,fit_intercept=True,solver='newton-cg',
                                             max_iter=5e8,multi_class='ovr',n_jobs=-1))
classification = make_pipeline(StandardScaler(),clf)

#classification=[]

X = np.array(TrainingData)
Y = np.array(TrainingLabel)
classification.fit(X,Y)
print(classification_report(classification.predict(X),Y))
print(classification.score(X,Y))
print('predict:',classification.predict(X)[0:200])
print('actual: ',Y[0:200])

ValueError: Solver newton-cg supports only l2 penalties, got l1 penalty.

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(penalty='l1',C=5,tol=10e-9,fit_intercept=True,solver='liblinear',
                                             max_iter=5e8,multi_class='ovr',n_jobs=-1))
classification = make_pipeline(StandardScaler(),clf)
n_samples=X.shape[0]
cv = ShuffleSplit(n_samples, n_iter=3,test_size=0.3, random_state=0)
scores = cross_val_score(classification, X, Y,cv=cv, scoring='accuracy')

print(scores)

In [ ]:
X_scale = scale(X)

In [ ]:
#
# transfer funtion
#
def sgm(x,Derivative = False):
    if not Derivative:
        return 1.0 / (1.0+np.exp(-x))
    else:
        out = sgm(x)
        return out*(1.0-out)
    
def linear(x,Derivative = False):
    if not Derivative:
        return x
    else:
        return 1.0
    
def gaussian(x, Derivative = False):
    if not Derivative:
        return np.exp(-x**2)
    else:
        return -2*x*np.exp(-x**2)
    
def tanh(x, Derivative = False):
    if not Derivative:
        return np.tanh(x)
    else:
        return 1 - np.tanh(x)**2
            

#
#Class
#


class BackPropagationNetwork:
    """A back-propagation Network"""
    
    #
    # Class members
    #
    layerCount = 0
    shape = None
    weights=[]
    # Family of functions
    tFuncs = []
    #
    # Class methods
    #
    def __init__(self, layerSize,layerFunctions = None):
        """ Initialize the network"""
        
        # layer info
        self.layerCount = len(layerSize) -1
        self.shape = layerSize
        
        if layerFunctions is None:
            lFuncs = []
            for i in range(self.layerCount):
                if i == self.layerCount - 1:
                    lFuncs.append(linear)
                else:
                    lFuncs.append(sgm)
        else:
            if len(layerSize) != len(layerFunctions):
                raise ValueError("Incompatible list of transfer functions.")
            elif layerFunctions[0] is not None:
                raise ValueError("Input layer cannot have a transfer function.")
            else:
                lFuncs = layerFunctions[1:]
                
                    
        self.tFuncs = lFuncs
        
        # Data from last Run
        self._layerInput = []
        self._layerOutput = []
        self._previousWeightDelta = []
        
        # Create the weight arrays
        for (l1,l2) in zip(layerSize[:-1],layerSize[1:]):
            self.weights.append(np.random.uniform(low=-0.5,high=0.5,size=(l2,l1+1)))
            self._previousWeightDelta.append(np.zeros((l2,l1+1)))

            
    #
    # Run method
    #
    def Run(self,input):
        """Run the network based on the input data"""
        
        lnCases = input.shape[0]
        
        # Clear out the previous intermediate value lists
        self._layerInput=[]
        self._layerOutput=[]
        
        # Run it!
        for index in range(self.layerCount):
            # Determine layer input
            if index ==0:
                layerInput = self.weights[0].dot(np.vstack([input.T,np.ones([1,lnCases])]))
            else:
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1],np.ones([1,lnCases])]))
                
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.tFuncs[index](layerInput))
        return self._layerOutput[-1].T
    
    #
    # TrainEpoch method
    #
    def TrainEpoch(self, input,target, trainingRate = 0.2,momentum = 0.5):
        """This method trains the network for one epoch"""
        
        delta = []
        lnCases = input.shape[0]
        
        # First run the network
        self.Run(input)
        
        # Calculate our deltas
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount -1:
                # compare to the target values
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta**2)
                delta.append(output_delta * self.tFuncs[index](self._layerInput[index],True))
            else:
                #Compare to the following layer's delta
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1,:] * self.tFuncs[index](self._layerInput[index],True))
                
            
        # Compute weight deltas
        for index in range(self.layerCount):
            delta_index = self.layerCount - 1 - index
            
            if index == 0:
                layerOutput = np.vstack([input.T, np.ones([1,lnCases])])
            else:
                layerOutput = np.vstack([self._layerOutput[index -1],np.ones([1,self._layerOutput[index-1].shape[1]])])
                
            curWeightDelta = np.sum(\
                                layerOutput[None,:,:].transpose(2,0,1) * delta[delta_index][None,:,:].transpose(2,1,0)\
                                ,axis = 0)
            weightDelta = trainingRate * curWeightDelta + momentum * self._previousWeightDelta[index]
            
            self.weights[index] -= weightDelta
            
            self._previousWeightDelta[index] = weightDelta
            
            
        return error
    
    # Transfer functions is moved outside the class
    
#
# if run as a script, create a test object
#
if __name__ == "__main__":
    #bpn = BackPropagationNetwork((2,2,1))
    #print(bpn.shape)
    #print(bpn.weights)
    
    lvInput = X_scale
    lvTarget = Y
    lFuncs = [None,sgm,sgm, gaussian]
    bpn = BackPropagationNetwork((X_scale.shape[1],8,2,1),lFuncs)
    lnMax = 100000
    lnErr = 1e-5
    for i in range(lnMax+1):
        err = bpn.TrainEpoch(lvInput, lvTarget,momentum=0.1)
        if i % 2500 == 0:
            print("Iteration {a}\tError: {b:0.6f}".format(a=i, b=err))
        if err <= lnErr:
            print("Desired error reached. Iter: {0}".format(i))
            break
    # Display output
    lvOutput = bpn.Run(lvInput)
    for i in range(lvInput.shape[0][40:60]):
        print("Input: {0}\nOutput: {1}".format(lvInput[i],lvOutput[i]))
    #print("Input: {0}\nOutput: {1}".format(lvInput,lvOutput))